# Data Extraction for OSMI data on Mental Health in the Tech Industry

This data is from Open Source Mental Illness (OSMI) using survey data from years 2014, 2016, 2017, 2018 and 2019. Each survey measures and attitudes towards mental health and frequency of mental health disorders in the tech workplace.

The relational database is set up with three tables: Survey, Question, and Answer.

Important notice: the data is *self-reported*, therefore a fair amount of manual cleaning is required, plus the validity of the data is questionable.

In [43]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [44]:
db_folder = "data"
db_filename = "mental_health.sqlite"

# Construct the relative file path
db_path = os.path.join(db_folder, db_filename)


In [45]:
con = sqlite3.connect(db_path)

# Check the available tables
tables = pd.read_sql("""
    SELECT * FROM sqlite_master 
    WHERE TYPE = 'table' """,con)

for row in tables["sql"]:
    print(row)

CREATE TABLE "Answer" (
	"AnswerText" VARCHAR(10000) NULL,
	"SurveyID" INTEGER NULL,
	"UserID" INTEGER NULL,
	"QuestionID" INTEGER NULL
)
CREATE TABLE "Question" (
	"questiontext" VARCHAR(1000) NULL,
	"questionid" INTEGER NULL
)
CREATE TABLE "Survey" (
	"SurveyID" INTEGER NOT NULL,
	"Description" VARCHAR(255) NULL,
	PRIMARY KEY ("SurveyID")
)


We have identified the 'SurveyID' and 'QuestionID' as being the connecting link within our tables. The webpage tells us the 'Answer' tabl contains the answers to the questions, and the 'Question' table contains the questions. Let's first check the number of respondents and then analyse what the 'Question' table looks like. 

In [46]:
respondents =pd.read_sql_query( """
        SELECT SurveyID AS Year, COUNT(DISTINCT UserID) AS RespondentsCount
        FROM Answer
        GROUP BY SurveyID
        ORDER BY 1   
    """, con)

print(respondents)

   Year  RespondentsCount
0  2014              1260
1  2016              1433
2  2017               756
3  2018               417
4  2019               352


There was no survey made in 2015, which is something we need to keep in mind. Let's see what were the questions.

In [47]:
questions = pd.read_sql_query("""SELECT * FROM Question""", con)

for id, row in questions.iterrows():
    print(row['questionid'], row['questiontext'])

questions.head()

1 What is your age?
2 What is your gender?
3 What country do you live in?
4 If you live in the United States, which state or territory do you live in?
5 Are you self-employed?
6 Do you have a family history of mental illness?
7 Have you ever sought treatment for a mental health disorder from a mental health professional?
8 How many employees does your company or organization have?
9 Is your employer primarily a tech company/organization?
10 Does your employer provide mental health benefits as part of healthcare coverage?
11 Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?
12 Would you bring up a mental health issue with a potential employer in an interview?
13 Is your primary role within your company related to tech/IT?
14 Do you know the options for mental health care available under your employer-provided health coverage?
15 Has your employer ever formally discussed mental health (for exampl

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5


According to the dataset owner, the questions were already grouped together for closeness, and the answers scores were standardized. Now we can proceed with taking the Answer table in a pandas dataframe for easier manipulation.

In [48]:
answers = pd.read_sql_query("""SELECT * FROM Answer""", con)

answers.head()

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1


At this point, it is time to ask the questions to work on. With a dataset on mental health in the tech industry, what are some interesting questions we can ask? And how can we answer them? An important aspect, given the questions we have available, are the demographics of the respondents. 

Let's say we want to let a client know what to work on if they had an idea on a mental health wellness program for their employees, who would benefit the most?

To answer to this, we need to dive into our data, and check along with the survey questions. Some interesting ideas are:

- How do mental health disorders and mental health support measures differ across age, sex, race and work position?
- Does working remotely or in office impact the frequency of mental health diagnoses? What about mental health support?
- How has the trend in diagnoses and support changed over the years? 
- Who would benefit the most from additional programs?
- What do individuals in the survey believe would improve mental health support in their companies?

For these answers, we need to mix responses from many questions together within a single table, ideally, then polish it to start working with it. Out of these questions, the most relevant for our research are:

- Q1 - What is your age?
- Q2 - What is your gender?
- Q10 - Does your employer provide mental health benefits as part of healthcare coverage?
- Q16 - Does your employer offer resources to learn more about mental health disorders and options for seeking help?
- Q33 - Do you currently have a mental health disorder?
- Q62 - Have you ever had a coworker discuss their or another coworker's mental health with you?
- Q89 - What is your race?
- Q94 - Do you know the options for mental health care your employer provides?
 


In [49]:
combined_data = pd.read_sql_query("""
SELECT
    UserID AS id,
    MAX(CASE WHEN QuestionID = 1 THEN AnswerText END) AS age,
    MAX(CASE WHEN QuestionID = 2 THEN AnswerText END) AS sex,
    MAX(CASE WHEN QuestionID = 10 THEN AnswerText END) AS benefits,
    MAX(CASE WHEN QuestionID = 16 THEN AnswerText END) AS resources,
    MAX(CASE WHEN QuestionID = 33 THEN AnswerText END) AS mhd,
    MAX(CASE WHEN QuestionID = 62 THEN AnswerText END) AS coworker_chat,
    MAX(CASE WHEN QuestionID = 89 THEN AnswerText END) AS race,
    MAX(CASE WHEN QuestionID = 93 THEN AnswerText END) AS support_options
FROM Answer
WHERE QuestionID IN (1, 2, 10, 16, 19, 33, 62, 89, 93)
GROUP BY UserID;
""", con, index_col='id')

combined_data.head()

,age,sex,benefits,resources,mhd,coworker_chat,race,support_options
id,,,,,,,,
1,37,Female,Yes,None,None,None,None,No
2,44,Male,Don't know,None,None,None,None,No
3,32,Male,No,None,None,None,None,No
4,31,Male,No,None,None,None,None,No
5,31,Male,Yes,None,None,None,None,Yes


In [50]:
combined_data.describe(include='all')

,age,sex,benefits,resources,mhd,coworker_chat,race,support_options
count,4218,4218,4218,2958,2958,1525,1525,1260
unique,65,101,6,4,5,3,11,2
top,30,Male,Yes,No,Yes,0,White,No
freq,250,2830,1744,1084,1237,674,873,884


In [51]:
# Iterate over each column to check the unique values
for col in combined_data:
    print(col)
    print(combined_data[col].unique())
    print()

age
['37' '44' '32' '31' '33' '35' '39' '42' '23' '29' '36' '27' '46' '41'
 '34' '30' '40' '38' '50' '24' '18' '28' '26' '22' '19' '25' '45' '21'
 '-29' '43' '56' '60' '54' '329' '55' '-1' '48' '20' '57' '58' '47' '62'
 '51' '65' '49' '5' '53' '61' '8' '11' '72' '52' '17' '63' '99' '323' '3'
 '66' '59' '15' '74' '70' '64' '67' '0']

sex
['Female' 'Male' 'Male-ish' 'Trans-female' 'something kinda male?'
 'queer/she/they' 'non-binary' 'Nah' 'All' 'Enby' 'fluid' 'Genderqueer'
 'Androgyne' 'Agender' 'Guy (-ish) ^_^' 'male leaning androgynous'
 'Trans woman' 'Neuter' 'Female (trans)' 'queer' 'A little about you' 'p'
 'ostensibly male, unsure what that really means' 'Bigender'
 'Female assigned at birth' 'fm' 'Transitioned, M2F'
 'Genderfluid (born female)' 'Other/Transfeminine'
 'Female or Multi-Gender Femme' 'Androgynous' 'male 9:1 female, roughly'
 '-1' 'Other' 'nb masculine' 'none of your business' 'genderqueer' 'Human'
 'Genderfluid' 'genderqueer woman' 'mtf' 'Queer' 'Fluid'
 'Male/gend

We need to do a lot of data cleaning here. Some noticeable things we need to fix are:

- Ages being incorrectly inputted
- A lot of different genders
- Many '-1' values in the columns, this likely signifies NAs.

Let's get started with the age column and then proceed by making NAs standard and leaving 3 values for gender, namely Male, Female and Others. After dealing with these we will work on transforming '-1's as nan, and remove repeated values (for example "I don't know" and "Don't know" are the same thing). 
We can do all of this in pandas since we have already extracted the query, but let's imagine that our client requires us to do this in SQL as well. This might turn really ugly if we do it all together like before, so instead we will do it in two steps for age and sex with SQL and leave the rest to pandas.

In [52]:
cleaned_data_age = pd.read_sql_query(
        """
        SELECT
            UserID as id, 
            CASE 
                WHEN CAST(AnswerText AS INT) <= -18 THEN CAST(AnswerText AS INT) * -1
                WHEN CAST(AnswerText AS INT) <= 18 OR CAST(AnswerText AS INT) > 80 THEN NULL 
                ELSE CAST(AnswerText AS INT)
             END as age
        FROM Answer
        WHERE QuestionID = 1;
        """, con, index_col='id')

# print all rows unique values
print(cleaned_data_age.unstack().unique())

# remove nans from column 'age;
cleaned_data_age = cleaned_data_age.dropna()
cleaned_data_age['age']

[37. 44. 32. 31. 33. 35. 39. 42. 23. 29. 36. 27. 46. 41. 34. 30. 40. 38.
 50. 24. nan 28. 26. 22. 19. 25. 45. 21. 43. 56. 60. 54. 55. 48. 20. 57.
 58. 47. 62. 51. 65. 49. 53. 61. 72. 52. 63. 66. 59. 74. 70. 64. 67.]


id
1       37.0
2       44.0
3       32.0
4       31.0
5       31.0
        ... 
4214    27.0
4215    48.0
4216    50.0
4217    30.0
4218    24.0
Name: age, Length: 4194, dtype: float64

In [53]:
cleaned_data_sex = pd.read_sql_query(
    """
    SELECT
    UserID as id,
        CASE 
            WHEN LOWER(AnswerText) = 'female' THEN 'Female'
            WHEN LOWER(AnswerText) = 'male' THEN 'Male'
            ELSE 'Others'
        END as sex
    FROM Answer
    WHERE QuestionID = 2;
    """, con, index_col='id')

cleaned_data_sex.value_counts()

sex   
Male      3043
Female    1024
Others     151
Name: count, dtype: int64

Now that both age and sex columns are ready to be used, let's make a copy of the original dataframe and combine the new columns with the original dataframe. We will remove the pre-existing age and sex columns, and add the new ones. Then we will elaborate the other columns to ensure cleaned data.

In [54]:
df_clean = combined_data.copy()

In [55]:
cols = ['benefits', 'resources', 'mhd', 'coworker_chat', 'support_options']

for col in cols:

    df_clean[col] = np.where(df_clean[col] == '0', 'No', df_clean[col])
    df_clean[col] = np.where(df_clean[col] == '1', 'Yes', df_clean[col])

    df_clean[col] = np.where(df_clean[col].isin(["Yes", "No"]) | df_clean[col].isna(), df_clean[col], "Don't know")


for col in df_clean.columns:

    if df_clean[col].isin(['-1', None, '']).any():
        df_clean[col] = df_clean[col].replace(['-1', None, ''], np.nan)

for col in df_clean.columns:
    print(col)
    print(df_clean[col].unique())
    print()


age
['37' '44' '32' '31' '33' '35' '39' '42' '23' '29' '36' '27' '46' '41'
 '34' '30' '40' '38' '50' '24' '18' '28' '26' '22' '19' '25' '45' '21'
 '-29' '43' '56' '60' '54' '329' '55' nan '48' '20' '57' '58' '47' '62'
 '51' '65' '49' '5' '53' '61' '8' '11' '72' '52' '17' '63' '99' '323' '3'
 '66' '59' '15' '74' '70' '64' '67' '0']

sex
['Female' 'Male' 'Male-ish' 'Trans-female' 'something kinda male?'
 'queer/she/they' 'non-binary' 'Nah' 'All' 'Enby' 'fluid' 'Genderqueer'
 'Androgyne' 'Agender' 'Guy (-ish) ^_^' 'male leaning androgynous'
 'Trans woman' 'Neuter' 'Female (trans)' 'queer' 'A little about you' 'p'
 'ostensibly male, unsure what that really means' 'Bigender'
 'Female assigned at birth' 'fm' 'Transitioned, M2F'
 'Genderfluid (born female)' 'Other/Transfeminine'
 'Female or Multi-Gender Femme' 'Androgynous' 'male 9:1 female, roughly'
 nan 'Other' 'nb masculine' 'none of your business' 'genderqueer' 'Human'
 'Genderfluid' 'genderqueer woman' 'mtf' 'Queer' 'Fluid'
 'Male/gender

In [56]:
df_clean = df_clean.drop(['age', 'sex'], axis=1)

df_clean = df_clean.join(cleaned_data_age, on='id')
df_clean = df_clean.join(cleaned_data_sex, on='id')

df_clean

,benefits,resources,mhd,coworker_chat,race,support_options,age,sex
id,,,,,,,,
1,Yes,NaN,NaN,NaN,NaN,No,37.0,Female
2,Don't know,NaN,NaN,NaN,NaN,No,44.0,Male
3,No,NaN,NaN,NaN,NaN,No,32.0,Male
4,No,NaN,NaN,NaN,NaN,No,31.0,Male
5,Yes,NaN,NaN,NaN,NaN,Yes,31.0,Male
...,...,...,...,...,...,...,...,...
4214,Don't know,Don't know,Don't know,No,NaN,NaN,27.0,Male
4215,Don't know,Don't know,No,Don't know,White,NaN,48.0,Male
4216,Don't know,Don't know,No,Don't know,NaN,NaN,50.0,Male


Let's now check for outliers in our data, however given the responses are almost all categorical we can only check for outliers in the numerical column of age.

In [57]:
df_clean = df_clean.dropna(subset=['age'])
df_clean.loc[:, 'age'] = df_clean['age'].astype(int)

print(df_clean.isna().sum())

benefits              0
resources          1246
mhd                1246
coworker_chat      2674
race               3207
support_options    2948
age                   0
sex                   0
dtype: int64


This is self-reported data, we expect there to be a lot of NA values. This sort of data is sensitive to any whatsoever manipulation, and we want to avoid dirtying it. Therefore, we will leave the NAs as they are, and work with what we have comparing the columns with age and sex when possible.

In [58]:
df_clean.describe(include='all')

,benefits,resources,mhd,coworker_chat,race,support_options,age,sex
count,4194,2948,2948,1520,987,1246,4194.000000,4194
unique,3,3,3,3,10,2,NaN,3
top,Yes,Don't know,Yes,No,White,No,NaN,Male
freq,1737,1165,1234,672,872,878,NaN,3028
mean,NaN,NaN,NaN,NaN,NaN,NaN,33.892704,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,8.039488,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,28.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,33.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,38.000000,NaN


After all this cleaning, we see many interesting aspects of our data already: 

- Firstly, the most common country where respondents live and work is the US;
- The most common gender is Male, with 3028 out of 4194 respondents;
- The most common race is White, with 872 out of 987 respondents;
- It seems more people report having a mental health disorders than those who don't;
- Around half of the people seems to not have chats with coworkers about mental health topics.

Lastly, we will include a dataframe containing the improvements the respondents believe could be made to improve their mental health in the workplace.

In [59]:
improvements = pd.read_sql_query(
"""
SELECT UserID as id, AnswerText as improvements
FROM Answer
WHERE QuestionID = 86
  AND AnswerText IS NOT NULL
  AND LENGTH(AnswerText) > 10  -- Adjust the minimum length as needed
  AND AnswerText NOT LIKE '%sdsdsdsdsd%'  -- Exclude specific patterns
  AND AnswerText NOT LIKE '%Not sure%'   -- Exclude specific patterns
  AND AnswerText NOT LIKE '%invalid_pattern%'  -- Exclude other invalid patterns
""",con, index_col='id')

print(improvements.head())

                                           improvements
id                                                     
2694                       They don't take it seriously
2695  raise awareness, talk about it to lessen the s...
2696  Education and awareness, statistics, add suppo...
2697       More support, less burnout and death marches
2698  I think tech is more internal and they don't r...


In [60]:
data_folder = "data/clean_data"

improvements.to_csv(data_folder + '/improvements.csv')
df_clean.to_csv(data_folder + '/cleaned_data.csv')

Thus far, we have cleaned the data and prepared it for further analysis and visualization using SQL and pandas. We will save the df now and the report continues on the next notebook 'data_EDA'.